In [ ]:
"""Формирует csv файл про авокадо"""

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Читаем данные из csv
# Переменная df является типом DataFrame
my_filepath = "../input/avocado-prices/avocado.csv"
df = pd.read_csv(my_filepath)
df.head()

In [ ]:
# Получаем отфильтрованные данные по матчам 2018 года
avocado_2018 = df[df.year == 2018]

print('completed')
avocado_2018.head()
#result_columns

In [ ]:
regions = avocado_2018.groupby('region').agg(
    **{
    'AveragePrice': pd.NamedAgg(column='AveragePrice', aggfunc='mean'),
    'year': pd.NamedAgg(column='year', aggfunc='mean')
    }
)
# округляем все значения до 2 символов после запятой
regions = regions.round(2)
regions.head()

In [ ]:
# Выведем график по средней цене в интерисующих нас регионах
plt.title("Средняя цена")
avocados_for_max = regions.loc[['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise']]
sns.barplot(x=avocados_for_max.index, y=avocados_for_max['AveragePrice'])
plt.show()

In [ ]:
#Создаем csv файл
regions.to_csv("output.csv", index=True)

In [ ]:
my_filepath_ned = "../input/nedvijimost/Nedvijimost.csv"
df_ned = pd.read_csv(my_filepath_ned)
df_ned.head()

In [ ]:
cols_with_missing = [col for col in df_ned.columns
                     if df_ned[col].isnull().any()]
print(cols_with_missing)

In [ ]:
from sklearn.impute import SimpleImputer
df_ned_copy = pd.DataFrame()
for col in cols_with_missing:
    df_ned_copy[col] = df_ned[col].isnull()
df_ned_copy.head()

my_imputer = SimpleImputer(strategy='median')
imputed = pd.DataFrame(my_imputer.fit_transform(df_ned_copy))
for col in cols_with_missing:
    df_ned[col] = df_ned_copy[col]
cols_with_missing = [col for col in df_ned.columns
                     if df_ned[col].isnull().any()]
print(cols_with_missing)

In [ ]:
# Get list of categorical variables
s = (df_ned.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_df_ned = df_ned.copy()
label_encoder = LabelEncoder()
for col in object_cols:
    label_df_ned[col] = label_encoder.fit_transform(df_ned[col])
df_ned = label_df_ned.copy()
df_ned.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from learntools.machine_learning.ex4 import *
from learntools.core import binder
from sklearn.model_selection import train_test_split

y = df_ned['Стоимость (т.руб.)']
feature_columns = ['Район', 'Количество комнат', 'Первый/Последний этаж', 'Общая площадь (м2)', 'Жилая площадь (м2)', 'Площадь кухни (м2)', 'Наличие агенства', 'Состояние']
X = df_ned[feature_columns]
iowa_model = DecisionTreeRegressor()
iowa_model.fit(X, y)

print("First in-sample predictions:", iowa_model.predict(X.head()))
print("Actual target values for those homes:", y.head().tolist())

binder.bind(globals())

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)
iowa_model = DecisionTreeRegressor(random_state=1)
iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)

print("Validation MAE: {:,.0f}".format(val_mae))

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
candidate_max_leaf_nodes = [ 100, 250, 500, 5000]
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)
print("\nНаилучшее количество узлов в дереве: %d" %(best_tree_size))
best_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
print(f'Лучшая модель: {best_model}')

